In [1]:
from pathlib import Path
import os
p = Path(os.path.abspath('')).resolve().parents[0]
sys.path.insert(1, str(p))
from idxutils import gen_in_network_links, JSONOpen
from tqdm import tqdm
import pandas as pd
import requests
import sys
from _utils import istarmap
import aiohttp
import asyncio
from idxutils import gen_in_network_links
import traceback as tb

'''
Get size of each file in MRF (step 2)
'''

'\nGet size of each file in MRF (step 2)\n'

In [2]:
async def get_network_files(session, url):
    url = "https://mrf.healthsparq.com/aetnacvs-egress.nophi.kyruushsq.com/prd/mrf/AETNACVS_I/ALICSI/" + url.strip("%0A").strip()
    try:
        async with session.head(url) as r:
            size = r.headers.get("Content-Length")
            return url, size

                    
    except aiohttp.ClientResponseError as e:
        print(e)
        return url, None


    except asyncio.TimeoutError:
        return url, None

    except aiohttp.ClientConnectionError:
        return url, None

    except KeyboardInterrupt:
        import os; os.exit()

    # except UnicodeDecodeError:
    #     tb.print_exc()        
    except Exception as e:
        tb.print_exc()
        print(e)
        return url, None

In [3]:
from tqdm.asyncio import tqdm

# Not entirely sure why I wrote two methods

# Pandas
async def main(file):
    df = pd.read_csv(file)
    df1 = pd.DataFrame(columns=["url", "size"])
    async with aiohttp.ClientSession(raise_for_status = True) as session:
        df1[["url", "size"]] = await tqdm.gather(*(get_network_files(session, url) for url in df["url"]), leave=True, position=0)
        return df1

# File
async def main_file(file):
    df1 = pd.DataFrame(columns=["url", "size"])
    with open(file, "r") as f:
        total = len(f.readlines())
        f.seek(0)
        async with aiohttp.ClientSession(raise_for_status = True) as session:
            df1[["url", "size"]] = await tqdm.gather(*(get_network_files(session, url) for url in f), leave=True, position=0)
            return df1

In [4]:
t = await main_file("C:\\Users\\adria\\github\\BountyScrapers\\12_quest_tic\\aetna_tools\\aetna_indexes.txt")

100%|██████████| 1788/1788 [00:09<00:00, 193.59it/s]


In [18]:
t

,url,size
0,https://mrf.healthsparq.com/aetnacvs-egress.no...,50960177
1,https://mrf.healthsparq.com/aetnacvs-egress.no...,55515410
2,https://mrf.healthsparq.com/aetnacvs-egress.no...,70543060
3,https://mrf.healthsparq.com/aetnacvs-egress.no...,70543060
4,https://mrf.healthsparq.com/aetnacvs-egress.no...,76625462
...,...,...
1783,https://mrf.healthsparq.com/aetnacvs-egress.no...,3946331017
1784,https://mrf.healthsparq.com/aetnacvs-egress.no...,3946331017
1785,https://mrf.healthsparq.com/aetnacvs-egress.no...,3946952694
1786,https://mrf.healthsparq.com/aetnacvs-egress.no...,3867494009


In [20]:
t = t.sort_values(by=["size"])
t.to_csv("Aetna_payers_sorted.csv", index=False)